# In which we create vectors of all categorical features before splitting for testing, and used 'days ago' instead of datetimes so we can run more models. The results were not great because we do not have a one-to-one relationship between ticket_id and asset_id.

#### Factors from SQL query: 'ticket_id', 'asset_id', 'root_cause', 'ticket_creation_reason',
####      'latitude', 'longitude', 'tilt', 'azimuth', 'ticket_origin',
#### 'service_partner', 'ticket_assigned_days_ago', 'ticket_closed_days_ago',
####       'installed_by', 'installed_days_ago'
#### Models compared: 'Logistic Regression', 'Nearest Neighbors', 'RBF SVM',
####         'Decision Tree', 'Random Forest', 'Neural Net', 'Bagging', 'AdaBoost',
####       'Gradient Boost'

In [1]:
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB, ComplementNB, MultinomialNB
from sklearn.linear_model import RidgeClassifier
from sklearn.neural_network import MLPClassifier
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, log_loss, f1_score, auc
from sklearn.pipeline import FeatureUnion
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn import tree
import numpy as np
import pandas as pd
from datetime import datetime
from imblearn.over_sampling import SMOTE
from sklearn.utils import resample
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-paper')
import pickle

/Users/humanperson/anaconda3/envs/learn2env/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


Get Data

In [12]:
df_all_shards_b = pd.read_csv('../data/all_shards_b.csv')

In [13]:
df_all_shards_b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5247 entries, 0 to 5246
Data columns (total 11 columns):
ticket_id                         3633 non-null float64
asset_id                          5247 non-null object
root_cause                        3633 non-null object
ticket_creation_reason            3616 non-null object
ticket_origin                     3632 non-null object
service_partner                   3633 non-null object
date_ticket_initially_assigned    3628 non-null object
latitude                          3633 non-null float64
longitude                         3633 non-null float64
installed_by                      3615 non-null object
installation_date                 1871 non-null object
dtypes: float64(3), object(8)
memory usage: 451.0+ KB


In [14]:
# df_all_shards_b.head()

Both ticket_id and asset_id need to be strings

In [15]:
df_all_shards_b[['ticket_id', 'asset_id']] = df_all_shards_b[['ticket_id', 'asset_id']].astype(object)
df_all_shards_b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5247 entries, 0 to 5246
Data columns (total 11 columns):
ticket_id                         3633 non-null object
asset_id                          5247 non-null object
root_cause                        3633 non-null object
ticket_creation_reason            3616 non-null object
ticket_origin                     3632 non-null object
service_partner                   3633 non-null object
date_ticket_initially_assigned    3628 non-null object
latitude                          3633 non-null float64
longitude                         3633 non-null float64
installed_by                      3615 non-null object
installation_date                 1871 non-null object
dtypes: float64(2), object(9)
memory usage: 451.0+ KB


In [16]:
df_all_shards_b.ticket_id.nunique()

3591

## Deal with duplicates and nulls. A lot of this data came from the time before Omnidian. It's missing information we consider relevant, so we'll drop those columns.

In [19]:
df_all_shards_b.drop_duplicates(inplace=True)
df_all_shards_b.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4193 entries, 0 to 5246
Data columns (total 11 columns):
ticket_id                         3591 non-null object
asset_id                          4193 non-null object
root_cause                        3591 non-null object
ticket_creation_reason            3574 non-null object
ticket_origin                     3590 non-null object
service_partner                   3591 non-null object
date_ticket_initially_assigned    3586 non-null object
latitude                          3591 non-null float64
longitude                         3591 non-null float64
installed_by                      3573 non-null object
installation_date                 1829 non-null object
dtypes: float64(2), object(9)
memory usage: 393.1+ KB


In [20]:
df_all_shards_b.isnull().any()

ticket_id                          True
asset_id                          False
root_cause                         True
ticket_creation_reason             True
ticket_origin                      True
service_partner                    True
date_ticket_initially_assigned     True
latitude                           True
longitude                          True
installed_by                       True
installation_date                  True
dtype: bool

In [26]:
df_all_shards_b.dropna(axis=0, how='any', inplace=True)
df_all_shards_b.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1792 entries, 0 to 5085
Data columns (total 11 columns):
ticket_id                         1792 non-null object
asset_id                          1792 non-null object
root_cause                        1792 non-null object
ticket_creation_reason            1792 non-null object
ticket_origin                     1792 non-null object
service_partner                   1792 non-null object
date_ticket_initially_assigned    1792 non-null object
latitude                          1792 non-null float64
longitude                         1792 non-null float64
installed_by                      1792 non-null object
installation_date                 1792 non-null object
dtypes: float64(2), object(9)
memory usage: 168.0+ KB


# Convert everthing to numbers for our machine to read.

### We want to use ticket_id to look things up later and will not encode it.

In [27]:
df_sans_ticket = df_all_shards_b.drop(['ticket_id', 'root_cause'], axis=1).copy()
df_sans_ticket.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1792 entries, 0 to 5085
Data columns (total 9 columns):
asset_id                          1792 non-null object
ticket_creation_reason            1792 non-null object
ticket_origin                     1792 non-null object
service_partner                   1792 non-null object
date_ticket_initially_assigned    1792 non-null object
latitude                          1792 non-null float64
longitude                         1792 non-null float64
installed_by                      1792 non-null object
installation_date                 1792 non-null object
dtypes: float64(2), object(7)
memory usage: 140.0+ KB


In [28]:
# List our categorical features
categoricals = list(df_sans_ticket.columns[(df_sans_ticket.dtypes.values == np.dtype('object'))])
categoricals

['asset_id',
 'ticket_creation_reason',
 'ticket_origin',
 'service_partner',
 'date_ticket_initially_assigned',
 'installed_by',
 'installation_date']

In [29]:
encoder = OneHotEncoder(categories='auto', handle_unknown='ignore')
encoder.fit(df_sans_ticket[categoricals])

OneHotEncoder(categorical_features=None, categories='auto', drop=None,
              dtype=<class 'numpy.float64'>, handle_unknown='ignore',
              n_values=None, sparse=True)

In [30]:
enc_cat = pd.DataFrame(encoder.transform(df_sans_ticket[categoricals]).toarray(),
                         columns=encoder.get_feature_names())
enc_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1792 entries, 0 to 1791
Columns: 3962 entries, x0_101111473 to x6_2016-09-06 00:00:00
dtypes: float64(3962)
memory usage: 54.2 MB


In [36]:
# enc_cat.insert(loc=0, column='ticket_id', value=df101_e.ticket_id)
enc_cat.head()

,x0_101111473,x0_101111478,x0_101111485,x0_101111489,x0_101111497,x0_101111499,x0_101111511,x0_101111513,x0_101111521,x0_101111528,...,x6_2016-04-19 00:00:00,x6_2016-04-27 00:00:00,x6_2016-05-11 00:00:00,x6_2016-05-18 00:00:00,x6_2016-06-29 00:00:00,x6_2016-07-01 00:00:00,x6_2016-07-09 00:00:00,x6_2016-08-03 00:00:00,x6_2016-08-09 00:00:00,x6_2016-09-06 00:00:00
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
# grab the other features
df_other = df_all_shards_b.drop(categoricals, axis=1).copy()
df_other.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1792 entries, 0 to 5085
Data columns (total 4 columns):
ticket_id     1792 non-null object
root_cause    1792 non-null object
latitude      1792 non-null float64
longitude     1792 non-null float64
dtypes: float64(2), object(2)
memory usage: 70.0+ KB


In [33]:
# put it back together
df_enc = df_other.join(enc_cat)
df_enc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1792 entries, 0 to 5085
Columns: 3966 entries, ticket_id to x6_2016-09-06 00:00:00
dtypes: float64(3964), object(2)
memory usage: 54.3+ MB


In [34]:
df_enc.isna().any().sum()

3962

In [35]:
df_enc.isnull().any()

ticket_id                 False
root_cause                False
latitude                  False
longitude                 False
x0_101111473               True
x0_101111478               True
x0_101111485               True
x0_101111489               True
x0_101111497               True
x0_101111499               True
x0_101111511               True
x0_101111513               True
x0_101111521               True
x0_101111528               True
x0_101111533               True
x0_101111565               True
x0_101111567               True
x0_101111573               True
x0_101111576               True
x0_101111589               True
x0_101111606               True
x0_101111614               True
x0_101111621               True
x0_101111625               True
x0_101111629               True
x0_101111640               True
x0_101111652               True
x0_101111662               True
x0_101111673               True
x0_101111679               True
                          ...  
x6_2015-

### It looks like the 0.0 from our binary encoding turned into nulls. We'll change them back.

In [37]:
df_enc.fillna(0.0, inplace=True)

In [38]:
df_enc.isna().any().sum()

0

## Train-test-split.

In [39]:
X_enc = df_enc.drop(['root_cause'], axis=1).copy()
y_enc = df_enc['root_cause']
X_train_enc, X_test_enc, y_train_enc, y_test_enc = train_test_split(X_enc, y_enc, random_state=42,
                                                                    test_size=0.2)

In [40]:
# X_train_enc.head()

,ticket_id,latitude,longitude,x0_101111473,x0_101111478,x0_101111485,x0_101111489,x0_101111497,x0_101111499,x0_101111511,...,x6_2016-04-19 00:00:00,x6_2016-04-27 00:00:00,x6_2016-05-11 00:00:00,x6_2016-05-18 00:00:00,x6_2016-06-29 00:00:00,x6_2016-07-01 00:00:00,x6_2016-07-09 00:00:00,x6_2016-08-03 00:00:00,x6_2016-08-09 00:00:00,x6_2016-09-06 00:00:00
1172,15203.0,37.413367,-121.873286,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5023,29218.0,34.146582,-118.841237,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5033,24031.0,33.695164,-117.385826,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5016,18741.0,35.718205,-120.865597,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
188,13425.0,32.804040,-117.055745,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
X_train_enc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1433 entries, 1172 to 1187
Columns: 3965 entries, ticket_id to x6_2016-09-06 00:00:00
dtypes: float64(3965)
memory usage: 43.4 MB


In [42]:
y_train_enc.shape

(1433,)

In [43]:
# X_test_enc.head()

,ticket_id,latitude,longitude,x0_101111473,x0_101111478,x0_101111485,x0_101111489,x0_101111497,x0_101111499,x0_101111511,...,x6_2016-04-19 00:00:00,x6_2016-04-27 00:00:00,x6_2016-05-11 00:00:00,x6_2016-05-18 00:00:00,x6_2016-06-29 00:00:00,x6_2016-07-01 00:00:00,x6_2016-07-09 00:00:00,x6_2016-08-03 00:00:00,x6_2016-08-09 00:00:00,x6_2016-09-06 00:00:00
685,9115.0,32.826304,-116.997396,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
236,6126.0,33.398961,-112.083460,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1118,3603.0,34.448303,-118.498772,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
473,12803.0,33.830443,-116.474820,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4766,18753.0,33.471713,-117.073093,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
X_test_enc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 359 entries, 685 to 1380
Columns: 3965 entries, ticket_id to x6_2016-09-06 00:00:00
dtypes: float64(3965)
memory usage: 10.9 MB


In [45]:
y_test_enc.shape

(359,)

# **********************************

### Modeling

In [46]:
random_state=42

In [47]:
names = ['Logistic Regression', 'Nearest Neighbors', 'RBF SVM',
         'Decision Tree', 'Random Forest', 'Neural Net', 'Bagging', 'AdaBoost',
         'Gradient Boost']

In [31]:
# lr = LogisticRegression(random_state=42, solver='lbfgs',  multi_class='multinomial', max_iter=10000)
# lr.fit(X_train_enc, y_train_enc)
# lr.score(X_test_enc, y_test_enc)

In [32]:
# lr = LogisticRegression(random_state=42, solver='sag',
#                         multi_class='multinomial', max_iter=10000)
# lr.fit(X_train_enc, y_train_enc)
# lr.score(X_test_enc, y_test_enc)

In [33]:
# lr = LogisticRegression(random_state=42, solver='saga',
#                         multi_class='multinomial', max_iter=10000)
# lr.fit(X_train_enc, y_train_enc)
# lr.score(X_test_enc, y_test_enc)

In [34]:
# try weeiging less common classes. This is not great right now because the test data may not have all classes.
# lr = LogisticRegression(random_state=42, class_weight='balanced', solver='sag',
#                         multi_class='multinomial', max_iter=10000)
# lr.fit(X_train_enc, y_train_enc)
# lr.score(X_test_enc, y_test_enc)

In [35]:
kn = KNeighborsClassifier()
kn.fit(X_train_enc, y_train_enc)
kn.score(X_test_enc, y_test_enc)

0.5818181818181818

In [36]:
svc_g = SVC(gamma=2, C=1)
svc_g.fit(X_train_enc, y_train_enc)
svc_g.score(X_test_enc, y_test_enc)

0.7212121212121212

In [37]:
dt = DecisionTreeClassifier(min_samples_leaf=30)
dt.fit(X_train_enc, y_train_enc)
dt.score(X_test_enc, y_test_enc)

0.6060606060606061

In [38]:
dt = DecisionTreeClassifier(min_samples_leaf=100)
dt.fit(X_train_enc, y_train_enc)
dt.score(X_test_enc, y_test_enc)

0.5818181818181818

In [39]:
rf = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)
rf.fit(X_train_enc, y_train_enc)
rf.score(X_test_enc, y_test_enc)

0.5818181818181818

In [40]:
rf = RandomForestClassifier(n_estimators=100,criterion='entropy')
rf.fit(X_train_enc, y_train_enc)
rf.score(X_test_enc, y_test_enc)

0.8242424242424242

In [41]:
rf = RandomForestClassifier(n_estimators=100,criterion='gini')
rf.fit(X_train_enc, y_train_enc)
rf.score(X_test_enc, y_test_enc)

0.8666666666666667

In [42]:
rf = RandomForestClassifier(n_estimators=200,criterion='entropy')
rf.fit(X_train_enc, y_train_enc)
rf.score(X_test_enc, y_test_enc)

0.8424242424242424

In [43]:
mlp = MLPClassifier(alpha=1, max_iter=1000)
mlp.fit(X_train_enc, y_train_enc)
mlp.score(X_test_enc, y_test_enc)

0.3939393939393939

In [44]:
mlp = MLPClassifier(max_iter=100)
mlp.fit(X_train_enc, y_train_enc)
mlp.score(X_test_enc, y_test_enc)

0.3939393939393939

In [45]:
mlp = MLPClassifier(max_iter=100)
mlp.fit(X_train_enc, y_train_enc)
mlp.score(X_test_enc, y_test_enc)

0.2

In [46]:
bag = BaggingClassifier(random_state=42)
bag.fit(X_train_enc, y_train_enc)
bag.score(X_test_enc, y_test_enc)

0.8242424242424242

In [47]:
bag = BaggingClassifier(random_state=42, bootstrap_features=True)
bag.fit(X_train_enc, y_train_enc)
bag.score(X_test_enc, y_test_enc)

0.8303030303030303

In [48]:
bag = BaggingClassifier(random_state=42, bootstrap_features=True, warm_start=True)
bag.fit(X_train_enc, y_train_enc)
bag.score(X_test_enc, y_test_enc)

0.8303030303030303

In [49]:
ada = AdaBoostClassifier(random_state=42)
ada.fit(X_train_enc, y_train_enc)
ada.score(X_test_enc, y_test_enc)

0.5818181818181818

In [50]:
gb = GradientBoostingClassifier(random_state=42, min_samples_leaf=30)
gb.fit(X_train_enc, y_train_enc)
gb.score(X_test_enc, y_test_enc)

0.7696969696969697